# MammoScan AI: 08 - Model Championship Playoff

## 🎯 Goal
This notebook is the final competition between our two best models: the retrained **Baseline CNN (v2)** and the **Regularized Transfer Learning Model (v2)**.

We will load each model, perform a full Precision-Recall analysis on the unseen test set, find their optimal classification thresholds, and compare their best possible performances side-by-side to declare a single, definitive champion model for our project.

In [11]:
# --- Core Libraries ---
# Import all necessary libraries for our analysis.
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, auc

# --- Path Setup ---
# This ensures our notebook can find the custom modules in `ml/src`.
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# --- Custom Modules ---
# Import our model-building functions. We need these to help load the models robustly.
from ml.src.model import build_full_model, create_regularized_transfer_model

# --- Constants ---
PROCESSED_DATA_DIR = os.path.join(project_root, 'data', 'processed')
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

## 📥 Step 1: Define Contenders and Load Test Data

First, we define the paths to our two "champion" model files that were trained with the corrected labels. We then load our `test` dataset and prepare two versions of it: one for our baseline model and one with the special preprocessing required by the EfficientNet transfer learning model.

In [12]:
# Ensure our processed data is available locally.
# !dvc pull data/processed.dvc

# --- Define Our Two Final Contenders ---
MODEL_PATHS = {
    "Baseline CNN (v2)": os.path.join(project_root, 'models', 'checkpoints', 'baseline_model_v2.keras'),
    "Transfer Learning (Regularized v2)": os.path.join(project_root, 'models', 'checkpoints', 'regularized_fine_tuned_model_v2.keras')
}

# --- Load the Test Dataset ---
print("Loading test dataset...")
test_dataset = tf.keras.utils.image_dataset_from_directory(
    os.path.join(PROCESSED_DATA_DIR, 'test'),
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='binary',
    shuffle=False # Never shuffle the test set during evaluation
)

# --- IMPORTANT: Remap labels to match our training (Cancer=1) ---
test_dataset = test_dataset.map(lambda x, y: (x, 1 - y))

# Create a second version of the dataset for the EfficientNet model
preprocess_input = tf.keras.applications.efficientnet.preprocess_input
test_dataset_preprocessed = test_dataset.map(lambda x, y: (preprocess_input(x), y))

# Get the true labels, which are the same for both dataset versions
true_labels = np.concatenate([y for x, y in test_dataset], axis=0)
class_names = ['Non-Cancer', 'Cancer'] # Class 0, Class 1
print("✅ Test data is ready.")

Loading test dataset...
Found 112 files belonging to 2 classes.
✅ Test data is ready.


2025-09-04 16:11:10.686908: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## 🏆 Step 2: The Championship Analysis Loop

Now for the main event. We'll loop through each contender. For each one, we'll use the specific loading strategy that we know works, get its predictions, and perform the Precision-Recall analysis to find its optimal score.

In [15]:
# CELL 6: The Championship Analysis Loop (Corrected)

# This list will hold the final results for each model
all_results = []

for model_name, model_path in MODEL_PATHS.items():
    print(f"\n--- Evaluating: {model_name} ---")
    
    # --- Step 2.1: Select the correct dataset ---
    if "baseline" in model_name:
        eval_dataset = test_dataset
    else:
        eval_dataset = test_dataset_preprocessed

    # --- Step 2.2: Load the Model using the ROBUST STRATEGY ---
    print("Re-creating model architecture from source code...")
    if "Baseline" in model_name:
        # For the baseline, we re-create the full model with its augmentation layers
        model = build_full_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    else: # For the transfer learning model
        # For the transfer model, we re-create its specific architecture
        model = create_regularized_transfer_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    
    print(f"Loading weights from {model_path}...")
    # Now, we load ONLY the weights into our clean, local architecture
    model.load_weights(model_path)
    print("✅ Model loaded successfully.")

    # --- Step 2.3: Get Predictions ---
    raw_predictions = model.predict(eval_dataset)
    
    # --- Step 2.4: P-R Curve Analysis ---
    precision, recall, thresholds = precision_recall_curve(true_labels, raw_predictions)
    
    pr_df = pd.DataFrame({
        'recall': recall[:-1],
        'precision': precision[:-1],
        'threshold': thresholds
    })

    # --- Step 2.5: Find Optimal Threshold for >= 90% Recall ---
    high_recall_options = pr_df[pr_df['recall'] >= 0.90]
    
    if not high_recall_options.empty:
        best_option = high_recall_options.loc[high_recall_options['precision'].idxmax()]
        
        # --- Step 2.6: Store the Results ---
        all_results.append({
            "Model": model_name,
            "Best Threshold": best_option['threshold'],
            "Recall": best_option['recall'],
            "Precision": best_option['precision'],
        })
        print("✅ Analysis complete.")
    else:
        print(f"⚠️ No threshold found for {model_name} that achieves at least 90% recall.")


--- Evaluating: Baseline CNN (v2) ---
Re-creating model architecture from source code...
Loading weights from /home/mr-rey/Joseph/Projects/Python/mammoscan-AI/models/checkpoints/baseline_model_v2.keras...


/home/mr-rey/miniconda3/envs/tensorflow/lib/python3.11/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:18: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


✅ Model loaded successfully.
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 355ms/step
✅ Analysis complete.

--- Evaluating: Transfer Learning (Regularized v2) ---
Re-creating model architecture from source code...
Loading weights from /home/mr-rey/Joseph/Projects/Python/mammoscan-AI/models/checkpoints/regularized_fine_tuned_model_v2.keras...
✅ Model loaded successfully.
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step
✅ Analysis complete.


## 🏁 Step 3: Final Results and Champion Declaration

Finally, we'll display our results in a clean table to make a direct, data-driven comparison and declare our champion model.

In [16]:
# Convert the results list into a pandas DataFrame for a clean summary table.
results_df = pd.DataFrame(all_results)

print("\n--- Model Championship Results ---")
print("Goal: Highest possible Precision while maintaining at least 90% Recall.")
display(results_df)

# Programmatically find and declare the champion.
if not results_df.empty:
    champion = results_df.loc[results_df['Precision'].idxmax()]
    print(f"\n🏆 CHAMPION MODEL: {champion['Model']}")
    print(f"It achieves a recall of {champion['Recall']:.2%} and precision of {champion['Precision']:.2%} at a threshold of {champion['Best Threshold']:.4f}")


--- Model Championship Results ---
Goal: Highest possible Precision while maintaining at least 90% Recall.


,Model,Best Threshold,Recall,Precision
0,Baseline CNN (v2),0.110593,0.947368,0.580645
1,Transfer Learning (Regularized v2),0.345100,0.947368,0.529412



🏆 CHAMPION MODEL: Baseline CNN (v2)
It achieves a recall of 94.74% and precision of 58.06% at a threshold of 0.1106
